In [38]:
from antlr4 import *

In [39]:
from AlteryxFunctionsTranspiler.AlteryxFunctionsLexer import AlteryxFunctionsLexer
from AlteryxFunctionsTranspiler.AlteryxFunctionsParser import AlteryxFunctionsParser
from AlteryxFunctionsTranspiler.AlteryxFunctionsVisitor import AlteryxFunctionsVisitor

In [40]:
from AlteryxFunctionsTranspiler.AlteryxToPython import AlteryxToPythonVisitor

In [41]:
def transpile_alteryx_to_python(alteryx_expression):
    input_stream = InputStream(alteryx_expression)
    lexer = AlteryxFunctionsLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = AlteryxFunctionsParser(stream)
    tree = parser.prog()

    visitor = AlteryxToPythonVisitor()
    return visitor.visit(tree)

### ALTERYX EXPRESSION: TONUMBER

In [42]:
alteryx_expression = 'ToNumber("42")'
python_code = transpile_alteryx_to_python(alteryx_expression)
print("Python Code:", python_code)
print("Execution Result:", eval(python_code))

Python Code: float("42")
Execution Result: 42.0


### ALTERYX EXPRESSION: IF TOBOOLEAN("1") THEN TOSTRING(100) ELSE "False" ENDIF

In [43]:
alteryx_expression = 'IF TOBOOLEAN("1") THEN TOSTRING(100) ELSE TONUMBER("200") ENDIF'
python_code = transpile_alteryx_to_python(alteryx_expression)
print("Python Code:", python_code)
print("Execution Result:", eval(python_code))

Python Code: (str(100) if None else float("200"))
Execution Result: 200.0


### ALTERYX EXPRESSION COMPLEX:

In [44]:
alteryx_expression = """
IF TOBOOLEAN("1")

    THEN
        TOSTRING(100)

    ELSE
        IF TOBOOLEAN("1")
            THEN ToNumber("42")

        ELSE ToNumber("49")

    ENDIF

ENDIF

"""


In [45]:
python_code = transpile_alteryx_to_python(alteryx_expression)
print("Python Code:", python_code)
print("Execution Result:", eval(python_code))

Python Code: (str(100) if None else (float("42") if None else float("49")))
Execution Result: 49.0
